<a href="https://colab.research.google.com/github/Jorgecuenca1/Tps_ProcesamientoLenguajeNatural/blob/main/3b_Correcci%C3%B2n_tp_clase_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2023-06-22 05:13:39--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2023-06-22 05:13:39--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2023-06-22 05:13:39--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['alicia-keys.txt',
 'kanye-west.txt',
 'adele.txt',
 'drake.txt',
 'bieber.txt',
 'bruno-mars.txt',
 'notorious-big.txt',
 'paul-simon.txt',
 'nursery_rhymes.txt',
 'bob-marley.txt',
 'joni-mitchell.txt',
 'Kanye_West.txt',
 'bjork.txt',
 'beatles.txt',
 'lil-wayne.txt',
 'kanye.txt',
 'disney.txt',
 'dj-khaled.txt',
 'prince.txt',
 'nirvana.txt',
 'patti-smith.txt',
 'nicki-minaj.txt',
 'al-green.txt',
 'blink-182.txt',
 'missy-elliott.txt',
 'leonard-cohen.txt',
 'bruce-springsteen.txt',
 'lin-manuel-miranda.txt',
 'jimi-hendrix.txt',
 'bob-dylan.txt',
 'johnny-cash.txt',
 'michael-jackson.txt',
 'nickelback.txt',
 'ludacris.txt',
 'radiohead.txt',
 'eminem.txt',
 'dickinson.txt',
 'janisjoplin.txt',
 'notorious_big.txt',
 'amy-winehouse.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'Lil_Wayne.txt',
 'lorde.txt',
 'britney-spears.txt',
 'rihanna.txt',
 'lady-gaga.txt',
 'dolly-parton.txt',
 'cake.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)
df.head()

<ipython-input-4-89d9bc44a57b>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/bob-marley.txt', sep='/n', header=None)


,0
0,"""Don't worry about a thing,"
1,'Cause every little thing gonna be all right.
2,"Singin': ""Don't worry about a thing,"
3,"'Cause every little thing gonna be all right!""..."
4,"Smiled with the risin' sun,"


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2218


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[["don't", 'worry', 'about', 'a', 'thing'],
 ["'cause", 'every', 'little', 'thing', 'gonna', 'be', 'all', 'right']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2218


In [12]:
# Cantidad de words encontradas en el corpus
num_words = len(w2v_model.wv)
print("Cantidad de palabras distintas en el corpus:", num_words)

Cantidad de palabras distintas en el corpus: 532


### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=4000,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 150431.25
Loss after epoch 1: 92639.9375
Loss after epoch 2: 91598.375
Loss after epoch 3: 91160.0
Loss after epoch 4: 89182.40625
Loss after epoch 5: 83465.03125
Loss after epoch 6: 76280.3125
Loss after epoch 7: 72554.0625
Loss after epoch 8: 68532.1875
Loss after epoch 9: 64487.9375
Loss after epoch 10: 62416.8125
Loss after epoch 11: 60083.625
Loss after epoch 12: 57400.0625
Loss after epoch 13: 52298.125
Loss after epoch 14: 51624.875
Loss after epoch 15: 51240.75
Loss after epoch 16: 49625.25
Loss after epoch 17: 49211.0
Loss after epoch 18: 48584.5
Loss after epoch 19: 47380.875
Loss after epoch 20: 47196.75
Loss after epoch 21: 46585.0
Loss after epoch 22: 46403.0
Loss after epoch 23: 46236.625
Loss after epoch 24: 45696.25
Loss after epoch 25: 45878.75
Loss after epoch 26: 45894.375
Loss after epoch 27: 44992.125
Loss after epoch 28: 44379.25
Loss after epoch 29: 44183.25
Loss after epoch 30: 44150.125
Loss after epoch 31: 44592.75
Loss after epoch 32: 4443

(43588870, 71852000)

### 4 - Ensayar

In [28]:
similar_words = w2v_model.wv.most_similar(positive=["worry"], topn=10)
for word, similarity in similar_words:
    print(f"{word}: {similarity}")


singin': 0.4555984139442444
fall: 0.42834582924842834
'bout: 0.4249691665172577
alright: 0.37303048372268677
happy: 0.3598896861076355
thing: 0.35635143518447876
message: 0.3505827784538269
pimper's: 0.3455803394317627
gun: 0.34260499477386475
little: 0.3294961750507355


In [29]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["worry"], topn=10)

[('roots', 0.018203111365437508),
 ('vibration', -0.012176807038486004),
 ('jah', -0.028318891301751137),
 ('word', -0.032436762005090714),
 ('first', -0.03879272937774658),
 ('too', -0.04466841742396355),
 ('fool', -0.048376649618148804),
 ('rastaman', -0.051271677017211914),
 ('came', -0.051643576472997665),
 ('on', -0.05538153648376465)]

In [30]:
from sklearn.decomposition import PCA
import plotly.express as px

similar_words = w2v_model.wv.most_similar(negative=["worry"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

In [31]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(positive=["happy"], topn=10)

[('cornerstone', 0.3770964443683624),
 ("cryin'", 0.3657260537147522),
 ('worry', 0.3598896861076355),
 ('ready', 0.3521519601345062),
 ('coffee', 0.3503662645816803),
 ('ask', 0.344417929649353),
 ('song', 0.337720662355423),
 ('blue', 0.31755074858665466),
 ('wrong', 0.315708190202713),
 ('message', 0.3150540590286255)]

In [32]:
from sklearn.decomposition import PCA
import plotly.express as px

similar_words = w2v_model.wv.most_similar(positive=["happy"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

In [33]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["happy"], topn=10)

[('some', 0.04305484890937805),
 ('been', 0.015359763987362385),
 ('may', 0.011414886452257633),
 ('god', 0.008973723277449608),
 ('ride', -0.0001873540022643283),
 ('rights', -0.0013440237380564213),
 ('too', -0.0016123566310852766),
 ("we're", -0.0024499220307916403),
 ('winepress', -0.0037879692390561104),
 ('down', -0.008398966863751411)]

In [34]:

similar_words = w2v_model.wv.most_similar(negative=["happy"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

In [35]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["carry"], topn=10)

[('can', 0.4591623842716217),
 ('top', 0.410715788602829),
 ('trodding', 0.41002213954925537),
 ('might', 0.39469975233078003),
 ('winepress', 0.3896510601043701),
 ('where', 0.3828642964363098),
 ('men', 0.3799769878387451),
 ('sun', 0.3798797130584717),
 ('refuse', 0.3764769732952118),
 ('understand', 0.37059125304222107)]

In [36]:
from sklearn.decomposition import PCA
import plotly.express as px

similar_words = w2v_model.wv.most_similar(positive=["carry"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

In [37]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["carry"], topn=10)

[('home', -0.014075775630772114),
 ('mean', -0.014894583262503147),
 ('must', -0.04160614311695099),
 ('own', -0.05171738564968109),
 ('dip', -0.054053258150815964),
 ('him', -0.05461045727133751),
 ('by', -0.0574447326362133),
 ('man', -0.06352297216653824),
 ('sheriff', -0.06425084918737411),
 ('rastaman', -0.06533803790807724)]

In [38]:


similar_words = w2v_model.wv.most_similar(negative=["carry"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

In [39]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["coming"], topn=5)

[('cold', 0.4345054030418396),
 ('after', 0.43127110600471497),
 ("we're", 0.39652013778686523),
 ('yard', 0.3895811438560486),
 ('mystic', 0.3873659372329712)]

In [40]:

similar_words = w2v_model.wv.most_similar(positive=["coming"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

In [41]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["coming"], topn=5)

[('live', -0.018225455656647682),
 ('fight', -0.03707129508256912),
 ('eh', -0.03959693759679794),
 ('walk', -0.040384840220212936),
 ('shed', -0.049502287060022354)]

In [42]:

similar_words = w2v_model.wv.most_similar(negative=["coming"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

In [43]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["could"])

[('over', 0.0013198801316320896),
 ('another', -0.02513829991221428),
 ('three', -0.026825806125998497),
 ('children', -0.02716630883514881),
 ('still', -0.028276905417442322),
 ('living', -0.032480400055646896),
 ('tonight', -0.036378975957632065),
 ('god', -0.03841937333345413),
 ('send', -0.04401835799217224),
 ('every', -0.044799793511629105)]

In [44]:

similar_words = w2v_model.wv.most_similar(negative=["could"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

In [45]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(positive=["could"])

[('loved', 0.5838057994842529),
 ('something', 0.5109092593193054),
 ('doo', 0.3822464048862457),
 ('light', 0.3619440495967865),
 ('air', 0.3482476472854614),
 ('blowing', 0.3387719392776489),
 ('matter', 0.32269126176834106),
 ('rock', 0.32117870450019836),
 ('fit', 0.32084429264068604),
 ('cap', 0.3127816617488861)]

In [46]:

similar_words = w2v_model.wv.most_similar(positive=["could"], topn=10)
words = [word for word, similarity in similar_words]

# Get word vectors for the selected words
word_vectors = [w2v_model.wv[word] for word in words]

# Apply PCA to reduce the dimensionality to 2
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors)

# Create a DataFrame with the embeddings and words
data = {"Word": words, "Dimension 1": embeddings_2d[:, 0], "Dimension 2": embeddings_2d[:, 1]}
df = pd.DataFrame(data)

# Create the scatter plot
fig = px.scatter(df, x="Dimension 1", y="Dimension 2", text="Word")

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings - Palabras que menos se relacionan con 'worry'",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()

### 5 - Visualizar agrupación de vectores

In [47]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [48]:
import plotly.graph_objects as go
import plotly.express as px
from sklearn.decomposition import PCA

# Get word vectors
word_vectors = w2v_model.wv

# Reduce dimensionality using PCA
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(word_vectors.vectors)

# Get word labels
labels = word_vectors.index_to_key

# Select a subset of words for visualization
MAX_WORDS = 200
embeddings_2d_subset = embeddings_2d[:MAX_WORDS]
labels_subset = labels[:MAX_WORDS]

# Create a scatter plot
fig = px.scatter(
    x=embeddings_2d_subset[:, 0],
    y=embeddings_2d_subset[:, 1],
    text=labels_subset,
    labels={"text": "Word"},
)

fig.update_traces(textposition="top center")

fig.update_layout(
    title="Word Embeddings",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    hovermode="closest",
)

fig.show()


### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.
similitudes en relación con la palabra "worry" en el modelo de palabras vectoriales (Word2Vec) utilizado. Las similitudes varían entre 0.3295 y 0.4556. Cada palabra tiene una similitud única que indica su grado de relación con "worry". En general, las palabras más cercanas en similitud son "singin'", "fall", "'bout" y "alright", mientras que las palabras menos relacionadas en términos de similitud son "happy", "thing", "message", "pimper's", "gun" y "little". La similitud se calcula en función de la proximidad de las palabras en el espacio vectorial de palabras del modelo.

Estas son las palabras que menos se relacionan con "worry" en el modelo de palabras vectoriales (Word2Vec):

'roots': 0.018203111365437508
'vibration': -0.012176807038486004
'jah': -0.028318891301751137
'word': -0.032436762005090714
'first': -0.03879272937774658
'too': -0.04466841742396355
'fool': -0.048376649618148804
'rastaman': -0.051271677017211914
'came': -0.051643576472997665
'on': -0.05538153648376465
Estas palabras tienen una relación menos cercana con "worry" en comparación con otras palabras del modelo. La similitud negativa indica que estas palabras están menos asociadas o menos relacionadas en términos de contexto y uso en comparación con la palabra "worry".
COMING:

Estas son las palabras que más se relacionan con "coming" en el modelo de palabras vectoriales (Word2Vec):

'cold': 0.4345054030418396
'after': 0.43127110600471497
"we're": 0.39652013778686523
'yard': 0.3895811438560486
'mystic': 0.3873659372329712
Estas palabras tienen una alta similitud con "coming" en términos de contexto y uso. Indican que estas palabras tienden a aparecer en contextos similares o se usan de manera similar a la palabra "coming".
Estas son las palabras que más se relacionan negativamente con "coming" en el modelo de palabras vectoriales (Word2Vec):

'live': -0.018225455656647682
'fight': -0.03707129508256912
'eh': -0.03959693759679794
'walk': -0.040384840220212936
'shed': -0.049502287060022354
Estas palabras tienen una baja similitud con "coming" en términos de contexto y uso. Indican que estas palabras tienden a aparecer en contextos distintos o se usan de manera diferente a la palabra "coming".
